In [1]:
import scipy.sparse as sp
import scipy.sparse.linalg as sp_linalg
import pandas as pd
from itertools import chain
import multiprocessing as mp
import numpy as np
import sys
import re

## Depending on the dataset (and the memory we have) you may use the following dtypes:

- np.ubyte: 1 byte Unsigned char ( 0 ... 255 )
- np.byte : 1 byte Signed char ( -128 ... 127 )
- np.short: 2 bytes C short   ( -32,768 ... 32,767 )  <-- This is more than enough

In [2]:
def getVec_sparse_v2(tx,elemList): #Tx = molecular formula, e.g. C4H2ClBr
    #### This regex handles non-integer subindices: C6H16Na3O12.5 (which happens in DS) 
    Li = re.split(r"(?<!^)(?=[A-Z])",tx)  #Split as ['H2','O']
    
    # Adds 1 if no subindex. Result is ['H2','O1']. 
    # Right after, split chem symbol from subindex as [['H',2],['O',1]]
    
    li = [re.split(r"([A-z]+)(([0-9]*[.])?[0-9]+)",i)
          if bool(re.match(r'[A-z]*([0-9]*[.])?[0-9]+',i))
          else re.split(r"([A-z]+)(([0-9]*[.])?[0-9]+)",i+'1') for i in Li]  
    
    # Construct two lists: input for sparse matrix construction
    col  = [elemList.index(i[1]) for i in li]  # Index of element i to put correspondent data
    data = [float(i[2]) for i in li]           # Num. atoms of element i
    
    for i in data:
        if float(i)!=int(i):
            return None  # Return empty lists or better None?
    return col,data

def getElems(DataFile,NMax=None):
    """NMax specifies number of rows of dataset to be used"""
    col_names = ['ID','formula','year']
    sep = '\t'

    df = pd.read_csv(DataFile,header=None,sep=sep,nrows=NMax,names=col_names)  #Load data
    
    df['formula'] = df['formula'].str.strip()   #Remove white spaces at begginning and end of string 

    elems = set([])

    for cmpnd in df['formula']:
        txt = ''.join(re.findall(r'[A-z]',cmpnd))   #Remove all subindices (there must be a regex to this but who knows)
        elems = elems.union(  set(re.split(r"(?<!^)(?=[A-Z])",txt))  )  # Add elements of this set to the set of known elements

    elems = sorted(list(elems)) # Convert to list and sort

    # Save this list of elements so it doesn't have to be calculated every time
    #with open("./Data/ElementList.txt", "w") as f:
    #    for A in elems:
    #        f.write(str(A) +"\n")

    return elems  # This returns a list with all sorted elements in dataset

def allVecs_sparse(DataFile,NMax=None):
    col_names = ['ID','formula','year']
    sep = '\t'

    df = pd.read_csv(DataFile,header=None,sep=sep,nrows=NMax,names=col_names)  #Load data
        
    df['formula'] = df['formula'].str.strip()   #Remove white spaces at begginning and end of string 

    elemList = getElems(DataFile,NMax)
    
    # List of lists [col,data]
    colXdata = list(map(lambda x: getVec_sparse_v2(x,elemList) , df['formula'].values))
    index = [i for i, l in enumerate(colXdata) if l is not None]
    colXdata = [l for l in colXdata if l is not None]
    
    # See docs for scipy.sparse.csr_matrix to understand the syntaxis
    indptr = np.cumsum([0]+list(map(lambda x: len(x[0]) , colXdata)))
    indices = np.array(list(chain(*[l[0] for l in colXdata])))
    data = np.array(list(chain(*[l[1] for l in colXdata])))

    cmpnds = sp.csr_matrix((data, indices, indptr), 
                           shape=(len(colXdata), len(elemList)),
                           dtype=np.short)
       
    years = df['year'].values[index]
    subsID = df['ID'].values[index]
    
    del indptr, indices, data, df, index, colXdata

    return cmpnds,years,subsID, elemList

In [3]:
sparse , years , subsID , elemList = allVecs_sparse('../SendProfGR/sample_w_IDs.csv')
sparse

<11138x60 sparse matrix of type '<class 'numpy.int16'>'
	with 41691 stored elements in Compressed Sparse Row format>

In [4]:
def findRs_sparse(cmpnds,elemList):
    indices = cmpnds.indices
    data = cmpnds.data
    indptr = cmpnds.indptr
    
    sz_cols = cmpnds.shape[1] # Number of elements
    
    Rs = []  
    cmpnd_num_Rs = []
    
    for c in range(indptr.shape[0]-1):
        indx = indices[indptr[c]:indptr[c+1]]
        sub_dat = data[indptr[c]:indptr[c+1]]
        
        for i in range(indx.shape[0]):
            c_data = sub_dat.copy()
            n = int(c_data[i])          
            
            for j in range(n):   #Loop through ith element's subindex
                c_data[i] -= 1   #Remove one
                n = j+1          #How many atoms of this element have been removed 

                #Append compound data with a reduced entry (R-X n-1)
                Rs.append(  np.append(c_data.copy(),n)  )

        cmpnd_num_Rs.append((sub_dat.sum(),indx))
   
    # Construct sparse matrix
    for_indics = list(chain(*[l[0]*[l[1]] for l in cmpnd_num_Rs]) )

    indptr = np.cumsum([0]+list(map(lambda x: len(x)+1 , for_indics)))
    indices = np.array(list(chain(*[list(l)+[sz_cols] for l in for_indics])))
    data = np.array(list(chain(*[l for l in Rs])))

    Rs = sp.csr_matrix((data, indices, indptr),
                        shape=(len(Rs), sz_cols+1),
                        dtype=np.short)

    return Rs

In [5]:
uniq_cmpnds_arr = np.unique(sparse.toarray(),axis=0)
uniq_sparse = sp.csr_matrix(uniq_cmpnds_arr,dtype=np.short)

years , subsID = years[:uniq_sparse.shape[0]] , subsID[:uniq_sparse.shape[0]] 

In [6]:
%%time
Rs = findRs_sparse(uniq_sparse,elemList)
Rs

CPU times: user 4.25 s, sys: 182 ms, total: 4.43 s
Wall time: 4.53 s


<211425x61 sparse matrix of type '<class 'numpy.int16'>'
	with 1102416 stored elements in Compressed Sparse Row format>

---
#### Next step: Finding which Rs are more than once in DS.

1. Distribute the total list of Rs in chunks of non-overlapping sets
    - That means, if a vector `v` is in chunk $i$, then all of its repetitions (if any) should as well be in chunk i and in no other chunk $j \ne i$.
    
    - That way we can take advantage of multiple processors.


    
2. Find non-unique vectors within each chunk.

In [7]:
import scipy
scipy.__version__

'1.2.1'

In [8]:
def validRs(Rs):
    """Get all non-unique R-n vectors out of a chunk Rs."""
    new_rs , c = np.unique(Rs.toarray(),axis=0, return_counts=True)
    new_rs = new_rs[c > 1]
    return sp.csr_matrix(new_rs,dtype=np.short)

def unique_mp(Rs,size=4):
    """Create data chunks for finding non-unique Rs in parallel.
    Test how this does on full DS. If pickling errors, implement recursive spliting.
    """
    max_n = 4
    # First split by n, the most natural choice.
    dist_list = [Rs[(Rs[:,-1]==i).toarray()[:,-1] ]
                  for i in range(1,max_n)]   + [Rs[(Rs[:,-1]>=max_n).toarray()[:,-1]]]

    with mp.Pool(processes=size) as pool:
        R_results = [pool.apply_async(validRs,args=(r,))
                     for r in dist_list]        

        Rs_get = [r.get() for r in R_results]

    return Rs_get

In [9]:
uniqRs = unique_mp(Rs,size=4)
uniqRs

[<2715x61 sparse matrix of type '<class 'numpy.int16'>'
 	with 11245 stored elements in Compressed Sparse Row format>,
 <1874x61 sparse matrix of type '<class 'numpy.int16'>'
 	with 7781 stored elements in Compressed Sparse Row format>,
 <560x61 sparse matrix of type '<class 'numpy.int16'>'
 	with 2150 stored elements in Compressed Sparse Row format>,
 <965x61 sparse matrix of type '<class 'numpy.int16'>'
 	with 3759 stored elements in Compressed Sparse Row format>]

In [10]:
elemDict = {}
for i,elem in enumerate(elemList):
    elemDict[i] = elem

In [15]:
def get_matches(Rs,cmpnds,years,subsID,elemDict):
    """Get matches. For each R(n) find all elements X such that compound R-Xn exists in dataset. 
    Build an element set for each R(n).
    """
    ns = Rs[:,-1].data
    R  = Rs[:,:-1]
    
    sumCmpnds = cmpnds.sum(axis=1)
    sumRaxis1 = np.array(    R.sum(axis=1).flatten() + ns    ).flatten()
    
    Matches = []
    for i,n in enumerate(ns):
        if i%1000==0:       print( f"\t{i}th R evaluated..." )
            
        r = R[i] #The actual R
        
        """Encode a condition to search only within a subset of compounds
        fulfulling certain conditions based on R"""
        # 1. R is contained in compound        
        cond1 = ((cmpnds - r.toarray())>=0).all(axis=1)
        # 2. sum of atoms in cmpnd == sum of atoms in R_ (sum(R) + n)
        cond2 = (sumCmpnds == sumRaxis1[i])
        
        cond = np.array(cond1 & cond2).flatten()  # Combine conditions
        subsetCmpnds = cmpnds[cond]  # Select subset of cmpnds
        curr_years = years[cond]
        curr_subsID = subsID[cond]
        
        cmpnds_no_R = (subsetCmpnds - r.toarray())
        
        # Now select only those cmpnds where residual is due to one element only (X_n)
        # Only useful for n!=1
        if n!=1:
            cond = np.array((cmpnds_no_R!=0).sum(axis=1)==1).flatten()
            subsetCmpnds = subsetCmpnds[cond]
            curr_years = curr_years[cond]
            curr_subsID = curr_subsID[cond]
            
        # At this point, subsetCmpnds contains all compounds that match with R(n).
        elemIndex = (subsetCmpnds - r.toarray()).nonzero()[1]
        curr_list = set(map(lambda x: elemDict[x]  , elemIndex))  # Map dict to above list of elems

        Matches.append( [curr_list, curr_years, curr_subsID] )

        
        
        ###########
        ## Deal with this after you get all data created above
        #Table_list.append(getTable(curr_list,curr_years,curr_subsID,useID=useID))      
        ###########
    return Matches      # [list_elems,list_years, list_ids] for each R(n)
    
for i in range(4): get_matches(uniqRs[i],uniq_sparse,years,subsID,elemDict)

	0th R evaluated...
	1000th R evaluated...
	2000th R evaluated...
	0th R evaluated...
	1000th R evaluated...
	0th R evaluated...
	0th R evaluated...


In [ ]:
def getCommonal(Rs,rank,cmpnds,years,subsID,elemDict,useID=False):
    """Get Commonalities. For each R(n) find all elements X such that compound R-Xn exists in dataset. 
    Build a list of these for each R(n)
    rank is the number of processors in which the operation is to be run"""

    
    if rank==0:   print("\n Finding commonalities (finding sets for each (R,n) pair)...\n")
    
    
    Comm_list = []
    R_list = []
    Table_list = []
    # Now generate the R representations on TP (TPR).

    j=0 #Counter for Rs with more than one appearence
    sumCmpnds = cmpnds.sum(axis=1)

    for i,R_ in enumerate(Rs):
        if i%1000==0 and rank==0:       print( f"\t{i}th R evaluated..." )

        n = R_[-1]  #Take subindex 
        R = R_[:-1] #The actual R
        curr_list = []

        # Encode a condition to search only within a subset of compounds fulfulling certain conditions based on R
        # 1. R is contained in compound
        cond1 = ((cmpnds - R) >= 0).all(axis=1)
        # 2. sum of atoms in cmpnd == sum of atoms in R_ (sum(R) + n)
        cond2 = (cmpnds.sum(axis=1) == R_.sum())

        subsetCmpnds = cmpnds[cond1 & cond2]  # Select subset of cmpnds
        curr_years = years[cond1 & cond2]
        curr_subsID = subsID[cond1 & cond2]

        cmpnds_no_R = (subsetCmpnds - R)
        # Now select only those cmpnds where residual is due to one element only (X_n)
        subsetCmpnds = subsetCmpnds[(cmpnds_no_R!=0).sum(axis=1)==1]
        curr_years = years[cond1 & cond2]
        curr_subsID = subsID[cond1 & cond2]

        cmpnds_no_R = (subsetCmpnds - R)
        # Now select only those cmpnds where residual is due to one element only (X_n)
        subsetCmpnds = subsetCmpnds[(cmpnds_no_R!=0).sum(axis=1)==1]
        curr_years = curr_years[(cmpnds_no_R!=0).sum(axis=1)==1]
        curr_subsID = curr_subsID[(cmpnds_no_R!=0).sum(axis=1)==1]

        if subsetCmpnds.shape[0] > 1:

            curr_list = list(map(lambda x: elemDict[x]  , (subsetCmpnds - R).nonzero()[1] ))
            Table_list.append(getTable(curr_list,curr_years,curr_subsID,useID=useID))
            Comm_list.append(curr_list)
            R_list.append(R_)
            j+=1

    if rank==0:    print("Saving...")
    table_list_arr = np.array(Table_list,dtype=np.intc)
    np.save(f'./Data/TablesID_NMax{NMax}_P{rank}.npy',table_list_arr)
    np.save(f'./Data/RVector_NMax{NMax}_P{rank}.npy',np.array(R_list,dtype=np.short))


    ### Make a function here that produces a new array exchanging ID for year, just a mapping
    if useID:
        if rank==0:  print("\n\tProducing array filled with years from ID array...")

        # Mapping going from ID to year
        mapping = dict(zip(list(subsID),list(years)))
        mapping[-1] = -1
        # Apply mapping 
        yearArray = np.vectorize(mapping.__getitem__)(table_list_arr)
        np.save(f'./Data/TablesYears_NMax{NMax}_P{rank}.npy',np.array(yearArray,dtype=np.short))

    return Comm_list,R_list  #This list contains lists (one for each R) of elements X such that R-X exist in dataset.

In [ ]:
def distribRs_forUnique(Rs,max_n):

    def split_chunk(chunk,i):
        maxSplit = 5  # Maximum number of subsplits you want 

        # Split using ith index:
        split = []
        step = 2
        for j in range(maxSplit):
            lower,upper = j*step,(j+1)*step
            if j < maxSplit-1:         tmp_splt = chunk[(chunk[:,i]>=lower) & (chunk[:,i]<upper)]  # Entries that are either j or j+1
            else:                      tmp_splt = chunk[chunk[:,i]>=lower]   # Entries that are maxSplit-1 or larger
            split.append(tmp_splt)

        # Now recursively further split here
        newList = []
        for l in split:
            if l.nbytes/1e6 > maxWeightArray:
                print("\t** Found a very large chunk! (Inside recursive function)")
                newList = newList + split_chunk(l,i+1)  # Further split l by next i
            elif l.shape[0]>1:      newList.append(l)  # Append only if chunk contains more than one entry
        #    else:   newList.append(l)  # Append only if chunk contains more than one entry

        return newList


    # First split by n, the most natural choice.
    dis_list = [Rs[Rs[:,-1]==i] for i in range(1,max_n)]

    print("\nStarting recursive splitting of Rs")

    new_chunks = []
    for l in dis_list:
        if l.nbytes/1e6 > maxWeightArray:
            print("\t** Found a very large chunk!")
            new_chunks = new_chunks + split_chunk(l,0)
        else:        new_chunks.append(l)

    print("Ended recursion")

    return new_chunks


In [141]:
uniq_cmpnds = np.unique(sparse.toarray(),axis=0)
Rs = findRs(uniq_cmpnds)

Rs,c=np.unique(Rs,axis=0,return_counts=True)
Rs[c>1].shape

(6114, 61)